In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


In [2]:
# Membaca data dari URL
data_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
data = pd.read_csv(data_url)

# Ekstrak bulan dan tahun dari kolom Date
data['Date'] = pd.to_datetime(data['Date'])
data['Month'] = data['Date'].dt.month
data['Year'] = data['Date'].dt.year


In [3]:
# Membuat Aplikasi Dash dan Memberi Judul

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Automobile Sales Statistics Dashboard",
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),
    
    # Dropdown untuk memilih jenis laporan
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select a report type',
        value='Select Statistics',
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    
    # Dropdown untuk memilih tahun
    dcc.Dropdown(
        id='select-year',
        options=[{'label': i, 'value': i} for i in range(1980, 2024)],
        placeholder='Select year',
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    
    # Div untuk menampilkan output
    html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
])

In [4]:
# Membuat Callback untuk Mengaktifkan/Nonaktifkan Dropdown Tahun

@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value')
)
def update_input_container(selected_statistics):
    if selected_statistics == 'Yearly Statistics':
        return False
    else:
        return True


In [5]:
# Membuat Callback untuk Menghasilkan Grafik

@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='select-year', component_property='value')]
)
def update_output_container(selected_statistics, selected_year):
    if selected_statistics == 'Recession Period Statistics':
        recession_data = data[data['Recession'] == 1]

        # Grafik 1: Penjualan mobil rata-rata per tahun selama periode resesi
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, x='Year', y='Automobile_Sales', title="Penjualan Mobil Rata-rata per Tahun Selama Resesi")
        )

        # Grafik 2: Rata-rata jumlah kendaraan terjual berdasarkan jenis kendaraan selama periode resesi
        avg_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(avg_sales, x='Vehicle_Type', y='Automobile_Sales', title="Rata-rata Jumlah Kendaraan Terjual Selama Resesi")
        )

        # Grafik 3: Total bagian pengeluaran iklan berdasarkan jenis kendaraan selama periode resesi
        ad_expenses = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(ad_expenses, names='Vehicle_Type', values='Advertising_Expenditure', title="Total Pengeluaran Iklan Selama Resesi")
        )

        # Grafik 4: Efek tingkat pengangguran terhadap penjualan kendaraan selama resesi
        unemployment_effect = px.bar(
            recession_data, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type', title="Efek Tingkat Pengangguran Terhadap Penjualan Kendaraan Selama Resesi"
        )
        R_chart4 = dcc.Graph(figure=unemployment_effect)

        return [
            html.Div(className='chart-item', children=[R_chart1]),
            html.Div(className='chart-item', children=[R_chart2]),
            html.Div(className='chart-item', children=[R_chart3]),
            html.Div(className='chart-item', children=[R_chart4])
        ]

    elif selected_statistics == 'Yearly Statistics' and selected_year:
        year_data = data[data['Year'] == selected_year]

        # Grafik 1: Total penjualan mobil bulanan untuk tahun yang dipilih
        monthly_sales = year_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(monthly_sales, x='Month', y='Automobile_Sales', title=f"Total Penjualan Mobil Bulanan pada {selected_year}")
        )

        # Grafik 2: Rata-rata jumlah kendaraan terjual berdasarkan jenis kendaraan pada tahun yang dipilih
        avg_vehicle_sales = year_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.bar(avg_vehicle_sales, x='Vehicle_Type', y='Automobile_Sales', title=f"Rata-rata Jumlah Kendaraan Terjual pada {selected_year}")
        )

        # Grafik 3: Total pengeluaran iklan untuk setiap jenis kendaraan pada tahun yang dipilih
        yearly_ad_expenses = year_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.pie(yearly_ad_expenses, names='Vehicle_Type', values='Advertising_Expenditure', title=f"Total Pengeluaran Iklan pada {selected_year}")
        )

        return [
            html.Div(className='chart-item', children=[Y_chart1]),
            html.Div(className='chart-item', children=[Y_chart2]),
            html.Div(className='chart-item', children=[Y_chart3])
        ]

if __name__ == '__main__':
    app.run_server(debug=True)
